#### Diabetes in Embedding Space

Plan - strt by looking at one sick, one healthy patients. Use pandas to join between the seperate files.

In [2]:
cd ..

/home/dsi/rgoldner/projects/cvc


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os, sys
import collections
import importlib
import matplotlib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

import anndata as ad
import scanpy as sc

SRC_DIR = "cvc"
# assert os.path.isdir(SRC_DIR), f"Cannot find src dir: {SRC_DIR}"
# sys.path.append(SRC_DIR)

from cvc import model_utils
from cvc import plot_utils
from child_diabetes_timepoints import diabetes_utils as du
from cvc import embbeding_wrapper as ew
from child_diabetes_timepoints import graph_utils as gu
from cvc import utils
from lab_notebooks.utils import TRANSFORMER, DEVICE, DATA_DIR
MODEL_DIR = os.path.join(SRC_DIR, "models")
sys.path.append(MODEL_DIR)

FILT_EDIT_DIST = True

PLOT_DIR = os.path.join(os.path.dirname(SRC_DIR), "plots/CDR3_data_plots")
if not os.path.isdir(PLOT_DIR):
    os.makedirs(PLOT_DIR)
PLOT_DIR

INFO:matplotlib.font_manager:Failed to extract font properties from /usr/share/fonts/google-noto-emoji/NotoColorEmoji.ttf: In FT2Font: Could not set the fontsize (error code 0x17)


'plots/CDR3_data_plots'

#### Load and Process data

In [5]:
DATA_FOLDER_DIR = "/dsi/scratch/home/dsi/romiDiabetes/"

# combine files according to control/case metadata
case_samples_dir = DATA_FOLDER_DIR + "case_samples.csv"
control_samples_dir = DATA_FOLDER_DIR + "control_samples.csv"

In [6]:
# Process metadata
case_samples = du.process_diabetes_metadata(case_samples_dir)
control_samples = du.process_diabetes_metadata(control_samples_dir)

/home/dsi/rgoldner/anaconda3/envs/tcrbert/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (0,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):
/home/dsi/rgoldner/projects/cvc/child_diabetes_timepoints/diabetes_utils.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  samples['Adaptive ID'] = samples['Adaptive ID'].astype(str).str.replace('\.0', '')
/home/dsi/rgoldner/projects/cvc/child_diabetes_timepoints/diabetes_utils.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  samples['Adaptive ID'] = samples['Adaptive ID'].astype(str).str.replace('\.0', '')


In [7]:
# Join data for each patient
cases_dataframes_dict = du.join_data(case_samples[:3], DATA_FOLDER_DIR)
control_dataframes_dict = du.join_data(control_samples[:3], DATA_FOLDER_DIR)

/home/dsi/rgoldner/anaconda3/envs/tcrbert/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (62) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):
/home/dsi/rgoldner/anaconda3/envs/tcrbert/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (62) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):
/home/dsi/rgoldner/anaconda3/envs/tcrbert/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (62) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):
/home/dsi/rgoldner/anaconda3/envs/tcrbert/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (62) have mixed types.Specify dtype option on import or set low_memory=False.
  if awa

In [8]:
from tqdm import tqdm
# apply the parse_tags function to all dataframes that divides the sample_catalog_tags column into multiple columns
def apply_parse_tags(cases_dataframes_dict):
    for patient_id in tqdm(cases_dataframes_dict.keys()):
        dataframes_list = cases_dataframes_dict[patient_id]
        for i, df in enumerate(dataframes_list):
            dataframes_list[i] = df.join(df['sample_catalog_tags'].apply(du.parse_tags))
            # change cdr3_amino_acid column name to Sequences
            dataframes_list[i].rename(columns={'cdr3_amino_acid': 'Sequences'}, inplace=True)
            # change j_gene column to be all strings
            dataframes_list[i]['j_gene'] = dataframes_list[i]['j_gene'].astype(str)
    return cases_dataframes_dict

cases_dataframes_dict = apply_parse_tags(cases_dataframes_dict)
control_dataframes_dict = apply_parse_tags(control_dataframes_dict)

100%|██████████| 3/3 [04:32<00:00, 90.83s/it]


#### Create and Display embeddings

##### Create embeddings for one patient

In [9]:
cases_dataframes_dict[0][0].Sequences

0           CASSLESAGWGETQYF
1              CASSPQGELGQYF
2               CASSLDFGDQYF
3         CASSLASRAGGGTDTQYF
4           CASSFPGLAMRDTQYF
                 ...        
104505       CASSTRQGVNTEAFF
104506     CASSYSKGGNYNSPLHF
104507        CASSLAGSKYGYTF
104510         CSARERGLETQYF
104511       CASSLRQVGNSPLHF
Name: Sequences, Length: 82716, dtype: object

In [10]:
control_dataframes_dict[0]

[       sample_name locus                                sample_catalog_tags  \
 0      310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 1      310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 2      310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 3      310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 4      310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 ...            ...   ...                                                ...   
 18798  310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 18799  310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 18800  310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 18801  310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 18802  310101_TCRB  TCRB  Age:02 Years,Biological Sex:Female,Ethnic Grou...   
 
                                      

#### ClusTCR

In [ ]:
import clustcr
from clustcr import Clustering
# from clustcr.visualization import as_cluster_graph

patient_seqs = cases_dataframes_dict[0][0].Sequences
patient_seqs.drop_duplicates(inplace=True)
# Cluster your TCR data
res = Clustering(method='two-step').fit(patient_seqs)

In [25]:
res_df = res.clusters_df

In [32]:
# group by cluster and count the number of sequences in each cluster
res_df.groupby('cluster').count().sort_values(by='CDR3', ascending=False).head(20)

,CDR3
cluster,
1,541
833,237
2927,219
2926,201
839,198
1972,197
2,194
834,192
2930,180
